# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'06-27-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'06-27-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-06-28 04:21:29,33.93911,67.709953,111592,4519,68288.0,38785.0,Afghanistan,286.660045,4.049573
1,NaN,NaN,NaN,Albania,2021-06-28 04:21:29,41.15330,20.168300,132512,2456,129990.0,66.0,Albania,4604.628536,1.853417
2,NaN,NaN,NaN,Algeria,2021-06-28 04:21:29,28.03390,1.659600,138465,3693,96335.0,38437.0,Algeria,315.762159,2.667100
3,NaN,NaN,NaN,Andorra,2021-06-28 04:21:29,42.50630,1.521800,13882,127,13697.0,58.0,Andorra,17966.737850,0.914854
4,NaN,NaN,NaN,Angola,2021-06-28 04:21:29,-11.20270,17.873900,38556,889,32791.0,4876.0,Angola,117.311768,2.305737


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/18/21,6/19/21,6/20/21,6/21/21,6/22/21,6/23/21,6/24/21,6/25/21,6/26/21,6/27/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,98734,98734,98734,103902,105749,107957,109532,111592,111592,111592
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,132484,132488,132490,132490,132496,132497,132499,132506,132509,132512
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,135219,135586,135821,136294,136679,137049,137403,137772,138113,138465


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/18/21,6/19/21,6/20/21,6/21/21,6/22/21,6/23/21,6/24/21,6/25/21,6/26/21,6/27/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,3934,3934,3934,4215,4293,4366,4452,4519,4519,4519
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2454,2454,2454,2454,2455,2455,2455,2455,2456,2456
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,3615,3624,3631,3641,3650,3660,3669,3678,3685,3693


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/18/21,6/19/21,6/20/21,6/21/21,6/22/21,6/23/21,6/24/21,6/25/21,6/26/21,6/27/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,63426,63875,64401,65071,65565,66102,66799,67183,67483,68288
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,129903,129910,129918,129918,129941,129955,129966,129975,129982,129990
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,94093,94336,94571,94822,95084,95331,95599,95878,96108,96335


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/18/21,6/19/21,6/20/21,6/21/21,6/22/21,6/23/21,6/24/21,6/25/21,6/26/21,6/27/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,7241,7241,7241,7242,7242,7244,7244,7244,7244,7244
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,21868,21868,21868,21901,21901,21921,21921,21945,21945,21945
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2345,2345,2345,2345,2345,2344,2344,2344,2344,2344


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/18/21,6/19/21,6/20/21,6/21/21,6/22/21,6/23/21,6/24/21,6/25/21,6/26/21,6/27/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,113,113,113,113,113,113,113,113,113,113
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,314,314,314,314,314,314,314,314,314,314
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,59,59,59,59,59,60,60,60,60,60


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
656,1001.0,Autauga,Alabama,US,2021-06-28 04:21:29,32.539527,-86.644082,7244,113,NaN,NaN,"Autauga, Alabama, US",12966.045571,1.559912
693,1075.0,Lamar,Alabama,US,2021-06-28 04:21:29,33.779950,-88.096680,1472,37,NaN,NaN,"Lamar, Alabama, US",10662.803332,2.513587
694,1077.0,Lauderdale,Alabama,US,2021-06-28 04:21:29,34.901719,-87.656247,9632,250,NaN,NaN,"Lauderdale, Alabama, US",10387.257492,2.595515


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,33624983,603966,0.0
India,30233183,395751,29251029.0
Brazil,18420598,513474,16160826.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,33624983,603966,0.0,33021017.0,2021-06-28 04:21:29,37.950547,-91.419547
India,30233183,395751,29251029.0,586403.0,2021-06-28 04:21:29,23.088275,81.806127
Brazil,18420598,513474,16160826.0,1746298.0,2021-06-28 04:21:29,-12.669522,-48.480493
France,5831972,111130,404395.0,5316447.0,2021-06-28 04:21:29,6.430808,-34.730285
Turkey,5409027,49576,5275231.0,84220.0,2021-06-28 04:21:29,38.963700,35.243300


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3815751,63589,0.0
Texas,2994389,52273,0.0
Florida,2365464,37772,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3815751,63589,0.0,3752162.0,2021-06-28 04:21:29,37.843962,-120.728594
Texas,2994389,52273,0.0,2942116.0,2021-06-28 04:21:29,31.660643,-98.653069
Florida,2365464,37772,0.0,2327692.0,2021-06-28 04:21:29,28.940755,-82.700744
New York,2114123,53664,0.0,2060459.0,2021-06-28 04:21:29,42.544151,-75.474183
Illinois,1390432,25632,0.0,1364800.0,2021-06-28 04:21:29,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1249304,24490,0.0
Arizona,Maricopa,558730,10269,0.0
Illinois,Cook,556715,10478,0.0
Florida,Miami-Dade,508464,6472,0.0
Texas,Harris,403276,6575,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1249304,24490,0.0,1224814.0,2021-06-28 04:21:29,34.308284,-118.228241,6037.0
Arizona,Maricopa,558730,10269,0.0,548461.0,2021-06-28 04:21:29,33.348359,-112.491815,4013.0
Illinois,Cook,556715,10478,0.0,546237.0,2021-06-28 04:21:29,41.841448,-87.816588,17031.0
Florida,Miami-Dade,508464,6472,0.0,501992.0,2021-06-28 04:21:29,25.611236,-80.551706,12086.0
Texas,Harris,403276,6575,0.0,396701.0,2021-06-28 04:21:29,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-25,111592,132506,137772,13882,38371,1263,4374587,224635,30457,650016,...,4716065,363580,108184,4,266952,14537,313520,6905,143215,45217
2021-06-26,111592,132509,138113,13882,38528,1263,4393142,224728,30499,650192,...,4734011,364859,108730,4,268349,15325,313520,6906,146031,46018
2021-06-27,111592,132512,138465,13882,38556,1263,4405247,224797,30528,650261,...,4748644,365866,109223,4,269635,15740,313721,6908,148568,46442


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-25,4519,2455,3678,127,883,42,91979,4505,910,10700,...,128330,5447,725,1,3038,74,3556,1357,1915,1721
2021-06-26,4519,2456,3685,127,887,42,92317,4506,910,10699,...,128353,5474,727,1,3055,74,3556,1358,1967,1725
2021-06-27,4519,2456,3693,127,889,42,92568,4508,910,10700,...,128364,5494,729,1,3068,76,3559,1360,2022,1736


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-25,67183,129975,95878,13697,32605,1221,3990289,216423,23765,637051,...,15718,337748,104007,3,248066,5949,307236,4005,119411,37604
2021-06-26,67483,129982,96108,13697,32696,1221,4010346,216489,23778,637238,...,15718,340352,104461,3,249039,6137,307236,4019,121965,37761
2021-06-27,68288,129990,96335,13697,32791,1221,4027510,216558,23781,637424,...,15722,342494,105006,3,250528,6319,307618,4025,124582,37817


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/18/21,6/19/21,6/20/21,6/21/21,6/22/21,6/23/21,6/24/21,6/25/21,6/26/21,6/27/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,7241,7241,7241,7242,7242,7244,7244,7244,7244,7244
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1468,1468,1468,1469,1469,1472,1472,1472,1472,1472
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,9603,9603,9603,9612,9612,9621,9621,9632,9632,9632
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,3133,3133,3133,3134,3134,3135,3135,3135,3135,3135
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,16278,16278,16278,16287,16287,16301,16301,16317,16317,16317


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-25,549934,71035,0,892236,347254,3814010,555971,349120,109636,49,...,866670,2992812,413317,24392,3835,679762,449983,163804,677252,61951
2021-06-26,549934,71035,0,892711,347254,3814890,556542,349120,109657,49,...,866738,2994213,413668,24392,3835,680060,450404,163804,677252,61951
2021-06-27,549934,71035,0,892711,347254,3815751,556775,349120,109682,49,...,866738,2994389,413950,24392,3835,679828,450404,163804,677252,61951


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-06-25        7244   21945    2344  2686   6967    1247   2255   14761   
2021-06-26        7244   21945    2344  2686   6967    1247   2255   14761   
2021-06-27        7244   21945    2344  2686   6967    1247   2255   14761   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-06-25         3733     1874  ...    2902    680     3265      797   
2021-06-26         3733     1874  ...    2902    680     3265      797   
2021-06-27         3733     1874  ...    2902    680     3265      797   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-06-25           4729  3801  2315          0      926    656  
2021-06-26           4729  3801  2315          0      926    656  
2021-06-27           4729  3801  2315          0      926    656  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-25,11336,374,0,17903,5893,63536,6771,8275,1693,0,...,12543,52246,2351,256,30,11396,5902,2876,8099,740
2021-06-26,11336,374,0,17914,5893,63569,6770,8275,1694,0,...,12550,52265,2355,256,30,11402,5902,2876,8099,740
2021-06-27,11336,374,0,17914,5893,63589,6777,8275,1694,0,...,12550,52273,2363,256,30,11396,5902,2876,8099,740


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-06-25         113     314      60   64    139      42     71     330   
2021-06-26         113     314      60   64    139      42     71     330   
2021-06-27         113     314      60   64    139      42     71     330   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-06-25          124       45  ...      30     12       31        7   
2021-06-26          124       45  ...      30     12       31        7   
2021-06-27          124       45  ...      30     12       31        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-06-25             40    11    13          0       26      6  
2021-06-26             40    11    13          0       26      6  
2021-06-27             40    11    13          0       26      6  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-25,0.018807,0.000053,0.002686,0.00036,0.007351,0.0,0.005522,0.000454,0.001085,0.000145,...,0.003271,0.004381,0.004419,0.0,0.004931,0.014941,0.001613,0.000000,0.018454,0.020562
2021-06-26,0.000000,0.000023,0.002475,0.00000,0.004092,0.0,0.004242,0.000414,0.001379,0.000271,...,0.003805,0.003518,0.005047,0.0,0.005233,0.054207,0.000000,0.000145,0.019663,0.017715
2021-06-27,0.000000,0.000023,0.002549,0.00000,0.000727,0.0,0.002755,0.000307,0.000951,0.000106,...,0.003091,0.002760,0.004534,0.0,0.004792,0.027080,0.000641,0.000290,0.017373,0.009214


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-25,0.015049,0.000000,0.002453,0.0,0.002270,0.0,0.005917,0.000000,0.0,0.000280,...,0.000140,0.006281,0.002766,0.0,0.004962,0.027778,0.000281,0.000737,0.032345,0.007022
2021-06-26,0.000000,0.000407,0.001903,0.0,0.004530,0.0,0.003675,0.000222,0.0,-0.000093,...,0.000179,0.004957,0.002759,0.0,0.005596,0.000000,0.000000,0.000737,0.027154,0.002324
2021-06-27,0.000000,0.000000,0.002171,0.0,0.002255,0.0,0.002719,0.000444,0.0,0.000093,...,0.000086,0.003654,0.002751,0.0,0.004255,0.027027,0.000844,0.001473,0.027961,0.006377


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-25,0.005749,0.000069,0.002918,0.000219,0.008194,0.0,0.005710,0.000513,0.000253,0.000201,...,0.000382,0.008290,0.005170,0.0,0.004385,0.032992,0.002297,0.002503,0.030311,0.002132
2021-06-26,0.004465,0.000054,0.002399,0.000000,0.002791,0.0,0.005026,0.000305,0.000547,0.000294,...,0.000000,0.007710,0.004365,0.0,0.003922,0.031602,0.000000,0.003496,0.021388,0.004175
2021-06-27,0.011929,0.000062,0.002362,0.000000,0.002906,0.0,0.004280,0.000319,0.000126,0.000292,...,0.000254,0.006293,0.005217,0.0,0.005979,0.029656,0.001243,0.001493,0.021457,0.001483


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-25,0.000983,0.0,NaN,0.000658,0.00087,0.000311,0.000724,0.000364,0.000146,0.0,...,0.000000,0.000543,0.000748,0.000287,0.016702,0.000546,0.001095,0.000366,0.000097,0.001682
2021-06-26,0.000000,0.0,NaN,0.000532,0.00000,0.000231,0.001027,0.000000,0.000192,0.0,...,0.000078,0.000468,0.000849,0.000000,0.000000,0.000438,0.000936,0.000000,0.000000,0.000000
2021-06-27,0.000000,0.0,NaN,0.000000,0.00000,0.000226,0.000419,0.000000,0.000228,0.0,...,0.000000,0.000059,0.000682,0.000000,0.000000,-0.000341,0.000000,0.000000,0.000000,0.000000


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State Alabama                                                       \
Admin2         Autauga   Baldwin Barbour Bibb    Blount   Bullock    Butler   
2021-06-25         0.0  0.001095     0.0  0.0  0.001437  0.000803  0.002222   
2021-06-26         0.0  0.000000     0.0  0.0  0.000000  0.000000  0.000000   
2021-06-27         0.0  0.000000     0.0  0.0  0.000000  0.000000  0.000000   

Province_State                                ... Wyoming                     \
Admin2           Calhoun  Chambers  Cherokee  ...    Park    Platte Sheridan   
2021-06-25      0.002445  0.000536 -0.000533  ...     0.0  0.001473  0.00092   
2021-06-26      0.000000  0.000000  0.000000  ...     0.0  0.000000  0.00000   
2021-06-27      0.000000  0.000000  0.000000  ...     0.0  0.000000  0.00000   

Province_State                                                              \
Admin2         Sublette Sweetwater    Teton     Uinta Unassigned  Washakie   
2021-06-25          0.0    0.00127  0.00079  0.004338        NaN -0.001079   
2021-06-26          0.0    0.00000  0.00000  0.000000        NaN  0.000000   
2021-06-27          0.0    0.00000  0.00000  0.000000        NaN  0.000000   

Province_State         
Admin2         Weston  
2021-06-25        0.0  
2021-06-26        0.0  
2021-06-27        0.0  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-25,0.000706,0.0,NaN,0.001566,0.000509,0.000551,0.000887,0.000121,0.000000,NaN,...,0.000000,0.000555,0.005991,0.0,0.0,0.001318,0.000678,0.0,0.000865,0.0
2021-06-26,0.000000,0.0,NaN,0.000614,0.000000,0.000519,-0.000148,0.000000,0.000591,NaN,...,0.000558,0.000364,0.001701,0.0,0.0,0.000527,0.000000,0.0,0.000000,0.0
2021-06-27,0.000000,0.0,NaN,0.000000,0.000000,0.000315,0.001034,0.000000,0.000000,NaN,...,0.000000,0.000153,0.003397,0.0,0.0,-0.000526,0.000000,0.0,0.000000,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler  Calhoun   
2021-06-25         0.0     0.0     0.0  0.0    0.0     0.0    0.0  0.00304   
2021-06-26         0.0     0.0     0.0  0.0    0.0     0.0    0.0  0.00000   
2021-06-27         0.0     0.0     0.0  0.0    0.0     0.0    0.0  0.00000   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-06-25          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2021-06-26          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2021-06-27          0.0      0.0  ...     0.0    0.0      0.0      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-06-25            0.0   0.0   0.0        NaN      0.0    0.0  
2021-06-26            0.0   0.0   0.0        NaN      0.0    0.0  
2021-06-27            0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-25,0.018582,0.000035,0.002682,0.000385,0.005046,2.970151e-10,0.005453,0.000433,0.000863,0.000141,...,0.003243,0.004693,0.004206,4.517509e-21,0.005253,0.018001,0.000872,0.000433,0.021329,0.018531
2021-06-26,0.009291,0.000029,0.002579,0.000193,0.004569,1.485075e-10,0.004847,0.000423,0.001121,0.000206,...,0.003524,0.004105,0.004626,2.258755e-21,0.005243,0.036104,0.000436,0.000289,0.020496,0.018123
2021-06-27,0.004646,0.000026,0.002564,0.000096,0.002648,7.425376e-11,0.003801,0.000365,0.001036,0.000156,...,0.003308,0.003433,0.004580,1.129377e-21,0.005018,0.031592,0.000539,0.000289,0.018934,0.013668


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-25,0.018161,0.000026,0.002493,4.589676e-19,0.003362,2.551495e-13,0.006028,0.000203,2.911967e-26,0.000450,...,0.000146,0.006392,0.002293,0.0,0.006000,0.025089,0.000199,0.000586,0.031963,0.006698
2021-06-26,0.009081,0.000217,0.002198,2.294838e-19,0.003946,1.275747e-13,0.004852,0.000213,1.455983e-26,0.000178,...,0.000163,0.005674,0.002526,0.0,0.005798,0.012545,0.000100,0.000662,0.029559,0.004511
2021-06-27,0.004540,0.000108,0.002185,1.147419e-19,0.003100,6.378736e-14,0.003785,0.000328,7.279917e-27,0.000136,...,0.000124,0.004664,0.002638,0.0,0.005027,0.019786,0.000472,0.001067,0.028760,0.005444


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-25,0.007585,0.000082,0.002823,0.000469,0.006150,0.000011,0.005475,0.000422,0.000235,0.000257,...,0.001043,0.008409,0.004682,1.084202e-19,0.006160,0.026542,0.001226,0.002723,0.024669,0.002193
2021-06-26,0.006025,0.000068,0.002611,0.000235,0.004470,0.000006,0.005251,0.000364,0.000391,0.000275,...,0.000522,0.008059,0.004523,5.421011e-20,0.005041,0.029072,0.000613,0.003110,0.023028,0.003184
2021-06-27,0.008977,0.000065,0.002486,0.000117,0.003688,0.000003,0.004765,0.000341,0.000259,0.000283,...,0.000388,0.007176,0.004870,2.710505e-20,0.005510,0.029364,0.000928,0.002301,0.022243,0.002334


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-25,0.000602,0.000172,NaN,0.000632,0.000966,0.000294,0.000818,0.000323,0.000165,1.786669e-141,...,0.000064,0.000513,0.000885,0.000266,0.008539,0.000431,0.001188,0.000410,0.000140,0.001428
2021-06-26,0.000301,0.000086,NaN,0.000582,0.000483,0.000262,0.000922,0.000161,0.000178,8.933346e-142,...,0.000071,0.000490,0.000867,0.000133,0.004269,0.000435,0.001062,0.000205,0.000070,0.000714
2021-06-27,0.000150,0.000043,NaN,0.000291,0.000241,0.000244,0.000671,0.000081,0.000203,4.466673e-142,...,0.000036,0.000275,0.000774,0.000066,0.002135,0.000047,0.000531,0.000103,0.000035,0.000357


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-06-25      0.000045  0.000715 -0.000055  0.000050  0.000869  0.000555   
2021-06-26      0.000023  0.000358 -0.000027  0.000025  0.000435  0.000278   
2021-06-27      0.000011  0.000179 -0.000014  0.000013  0.000217  0.000139   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-06-25      0.001394  0.001242  0.000372 -0.000131  ...  0.000521   
2021-06-26      0.000697  0.000621  0.000186 -0.000066  ...  0.000260   
2021-06-27      0.000349  0.000311  0.000093 -0.000033  ...  0.000130   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-06-25      0.001707  0.001018 -0.001093   0.001742  0.000403  0.002185   
2021-06-26      0.000853  0.000509 -0.000546   0.000871  0.000202  0.001092   
2021-06-27      0.000427  0.000255 -0.000273   0.000436  0.000101  0.000546   

Province_State                                     
Admin2         Unassigned  Washakie        Weston  
2021-06-25           -1.0 -0.000471  5.873124e-09  
2021-06-26           -1.0 -0.000236  2.936562e-09  
2021-06-27           -1.0 -0.000118  1.468281e-09  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-25,0.000562,0.000335,NaN,0.001021,0.000546,0.000720,0.001188,0.000165,0.000468,NaN,...,0.000203,0.000574,0.003368,9.350038e-10,0.000051,0.001035,0.001866,0.000592,0.001146,0.001028
2021-06-26,0.000281,0.000168,NaN,0.000818,0.000273,0.000620,0.000520,0.000082,0.000529,NaN,...,0.000381,0.000469,0.002534,4.675019e-10,0.000026,0.000781,0.000933,0.000296,0.000573,0.000514
2021-06-27,0.000141,0.000084,NaN,0.000409,0.000136,0.000467,0.000777,0.000041,0.000265,NaN,...,0.000190,0.000311,0.002966,2.337510e-10,0.000013,0.000127,0.000466,0.000148,0.000286,0.000257


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State       Alabama                                                  \
Admin2               Autauga   Baldwin   Barbour          Bibb        Blount   
2021-06-25      1.944908e-08  0.000013  0.002119  1.412652e-17  1.667864e-15   
2021-06-26      9.724539e-09  0.000006  0.001059  7.063262e-18  8.339318e-16   
2021-06-27      4.862270e-09  0.000003  0.000530  3.531631e-18  4.169659e-16   

Province_State                                                                \
Admin2               Bullock        Butler   Calhoun  Chambers      Cherokee   
2021-06-25      5.546046e-15  1.372640e-11  0.002308  0.000008  1.541599e-22   
2021-06-26      2.773023e-15  6.863201e-12  0.001154  0.000004  7.707994e-23   
2021-06-27      1.386512e-15  3.431601e-12  0.000577  0.000002  3.853997e-23   

Province_State  ...       Wyoming                                        \
Admin2          ...          Park    Platte      Sheridan      Sublette   
2021-06-25      ...  8.705818e-31  0.011364  6.575178e-33  2.888535e-53   
2021-06-26      ...  4.352909e-31  0.005682  3.287589e-33  1.444267e-53   
2021-06-27      ...  2.176454e-31  0.002841  1.643795e-33  7.221337e-54   

Province_State                                                    \
Admin2         Sweetwater         Teton         Uinta Unassigned   
2021-06-25       0.000013  1.695421e-06  9.251859e-18   0.752941   
2021-06-26       0.000006  8.477105e-07  4.625929e-18   0.752941   
2021-06-27       0.000003  4.238553e-07  2.312965e-18   0.752941   

Province_State                              
Admin2              Washakie        Weston  
2021-06-25      1.794695e-45  1.525879e-06  
2021-06-26      8.973477e-46  7.629395e-07  
2021-06-27      4.486738e-46  3.814697e-07  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN
